In [57]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [59]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [65]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (304.66 seconds): Collecting URLs... indiatimes          

n_total     25732
n_unique    25672
dtype: int64

,pubs,actual_pub
https://businessinsider.co.za/boyfriend-stokvel-whatsapp-group-allows-you-to-advertise-your-ex-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/firing-of-chinese-facebook-engineer-prompts-anger-in-china-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/trump-fending-off-impeachment-with-greatest-economy-claims-economists-dispel-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/trump-ratchets-up-the-pressure-on-fox-news-ripping-it-apart-2019-10,"businessinsider, news24",businessinsider
https://cnbc.com/2019/10/11/trump-gives-his-administration-broad-powers-to-put-sanctions-on-turkey.html,"cnbc, realclearpolitics",cnbc
https://cnn.com/2019/10/11/media/fox-news-shepard-smith/index.html,"cnn, axios",cnn
https://finance.yahoo.com/news/u-china-said-reach-partial-190854141.html,"yahoo, realclearpolitics",yahoo
https://foxnews.com/opinion/trump-hispanic-voters-economy-democrats-mercedes-schlapp,"fox, realclearpolitics",fox
https://hotair.com/archives/jazz-shaw/2019/10/12/cuomo-orders-utility-pump-imaginary-natural-gas,"townhall, hotair",hotair
https://msnbc.com/hallie-jackson/watch/appeals-court-upholds-democrats-subpoena-for-trump-financial-records-71067717523,"msnbc, nbcnews",msnbc


huffpost                  82
cnn                      845
investing.com            934
politico                 233
time                      21
cnbc                     162
fox                      332
bbc                      269
businessinsider          559
morningstar               99
wsj                      109
nytimes                  108
guardian                 161
reuters                 1157
washingtontimes          470
washingtonpost           127
cbs                      405
marketwatch              174
atlantic                 103
vice                     101
npr                      886
newrepublic               20
yahoo                    330
independent              711
heritage                 280
zdnet                    395
townhall                 615
abcnews                  104
hotair                    84
cbc                      288
                        ... 
metro                    248
msnbc                    104
nationalreview           741
news24        

In [66]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [67]:
select_results = crawler.select(); display(*select_results)

100.00% (34.42 seconds): Selecting URLs... cryptonews          

n_total     464
n_unique    464
dtype: int64

,pubs,actual_pub


huffpost                 2
cnn                      9
investing.com           13
politico                 0
time                     0
cnbc                     1
fox                      7
bbc                      5
businessinsider         20
morningstar              1
wsj                      1
nytimes                  1
guardian                 8
reuters                 28
washingtontimes         11
washingtonpost          10
cbs                      0
marketwatch              2
atlantic                 0
vice                     0
npr                     16
newrepublic              0
yahoo                   63
independent              8
heritage                 0
zdnet                    0
townhall                 2
abcnews                  0
hotair                   4
cbc                      5
                        ..
metro                   12
msnbc                    1
nationalreview           0
news24                   9
techcrunch               1
arstechnica              1
e

In [68]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [ ]:
download_results = crawler.download(); download_results

96.55% (394.77 seconds): Downloading... yahoo               

In [14]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

[]